In [1]:
from pydantic_ai import Agent, RunContext, Tool
from pydantic_ai.models.ollama import OllamaModel
from pydantic import BaseModel, Field
from typing import Dict, List, Optional
from datetime import date

from dataclasses import dataclass

import nest_asyncio
nest_asyncio.apply()

In [2]:
model = OllamaModel(
    model_name='qwen2.5:7b',  
    base_url='http://localhost:11434/v1/',  
)

In [3]:
# Define Invoice schema
class Invoice(BaseModel):
    """Structure for invoice details."""
    invoice_id: str
    date_issued: date
    due_date: Optional[date]
    amount: float
    status: str  # e.g., "Paid", "Outstanding", "Overdue"

# Define Account Details schema
class AccountDetails(BaseModel):
    """Structure for account and billing-related details."""
    account_id: str
    customer_name: str
    email: str
    recent_invoices: Optional[List[Invoice]]
    outstanding_invoices: Optional[List[Invoice]]




In [4]:
# Define the structured response schema; The agent should respond to the client in this way
class ResponseModel(BaseModel):
    """Structured response with metadata and account details."""
    response: str  # The main response text
    needs_escalation: bool = False
    follow_up_required: bool =False
    sentiment: str = Field(description="Customer sentiment analysis")
    account_summary: Optional[AccountDetails] = None

In [5]:
# Example Customer account
customer_account = AccountDetails(
    account_id="A12345",
    customer_name="John Doe",
    email="john.doe@example.com",
    recent_invoices=[
        Invoice(
            invoice_id="INV001",
            date_issued=date(2025, 1, 10),
            due_date=date(2025, 1, 15),
            amount=120.0,
            status="Paid",
        )
    ],
    outstanding_invoices=[
        Invoice(
            invoice_id="INV002",
            date_issued=date(2025, 1, 5),
            due_date=date(2025, 1, 20),
            amount=150.0,
            status="Outstanding",
        )
    ],
)


In [6]:

# Agent with structured output and dependencies
agent5 = Agent(
    model=model,
    retries=5,
    result_type=ResponseModel,
    deps_type=AccountDetails,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "take in the client data and provide invoice information"
    ), 
)

In [7]:
# Add dynamic system prompt based on dependencies
@agent5.system_prompt
async def add_customer_details(ctx: RunContext[AccountDetails]) -> str:
    return f"Customer Acount details: {ctx.deps}"

In [8]:
# Query the agent for account information
response = agent5.run_sync(user_prompt="Can you tell me if i have any oustanding invoices?", deps=customer_account)
# Output the response
print(response.data.model_dump_json(indent=2))

/var/folders/nq/y2lyg7p15txfm5ksrw6f90340000gn/T/ipykernel_27482/3971403337.py:2: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  response = agent5.run_sync(user_prompt="Can you tell me if i have any oustanding invoices?", deps=customer_account)


{
  "response": "Yes, you have an outstanding invoice. The outstanding invoice is for the amount of $150.00 with a due date of January 20, 2025.",
  "needs_escalation": false,
  "follow_up_required": false,
  "sentiment": "neutral",
  "account_summary": {
    "account_id": "A12345",
    "customer_name": "John Doe",
    "email": "john.doe@example.com",
    "recent_invoices": [],
    "outstanding_invoices": [
      {
        "invoice_id": "INV002",
        "date_issued": "2025-01-05",
        "due_date": "2025-01-20",
        "amount": 150.0,
        "status": "Outstanding"
      }
    ]
  }
}


In [9]:
response.all_messages()
print(response.data.model_dump_json(indent=2))



{
  "response": "Yes, you have an outstanding invoice. The outstanding invoice is for the amount of $150.00 with a due date of January 20, 2025.",
  "needs_escalation": false,
  "follow_up_required": false,
  "sentiment": "neutral",
  "account_summary": {
    "account_id": "A12345",
    "customer_name": "John Doe",
    "email": "john.doe@example.com",
    "recent_invoices": [],
    "outstanding_invoices": [
      {
        "invoice_id": "INV002",
        "date_issued": "2025-01-05",
        "due_date": "2025-01-20",
        "amount": 150.0,
        "status": "Outstanding"
      }
    ]
  }
}
